In [1]:
import numpy as np

In [2]:
def xyz_read(filename):
    coordinates = []
    f = open(filename, "r")
    adj = [[], [], []]
    n_atoms = int(f.readline())
    m_name = f.readline()
    for line in f:
        atom, x, y, z, n, n1, n2, n3 = line.split()
        coordinates.append([float(x), float(y), float(z)])
        adj[0].append(int(n1) - 1)
        adj[1].append(int(n2) - 1)
        adj[2].append(int(n3) - 1)
    f.close()

    return n_atoms, np.array(coordinates), np.array(adj)

In [3]:
def dr(x, y):
    return coo[x] - coo[y]

In [4]:
# find the first polygen
def find_polygen(adj, Np):
    Np = 5
    for x0 in range(n):
        for x1 in adj[:, x0]:
            for xx in adj[:, x1]:
                if xx != x0:
                    x2 = xx

                    lorder = [x0, x1, x2]
                    flag = [0 for i in range(n)]
                    for x in lorder:
                        flag[x] = 1
                    while 1:
                        dr1 = dr(lorder[-2], lorder[-3])
                        dr2 = dr(lorder[-1], lorder[-2])
                        cr0 = np.cross(dr2, dr1)
                        watch = 0
                        for x in adj[:, lorder[-1]]:
                            if (cr0 @ np.cross(dr(x, lorder[-1]), dr2) > 0) & (
                                flag[x] == 0
                            ):
                                lorder.append(x)
                                flag[x] = 1
                            else:
                                watch += 1
                        if watch == 3:
                            break
                    nlayer = 1
                    if np.size(lorder) == Np:
                        break
            if np.size(lorder) == Np:
                break
        if np.size(lorder) == Np:
            break

    return lorder

In [5]:
def find_loop_contraction(adj, polygen):
    lorder = polygen.copy()
    nlayer = 1
    flag = [0 for i in range(n)]
    for x in lorder:
        flag[x] = 1
    Contract_list = []
    Contract221_list = []
    while lorder != []:
        Contract_list.append(lorder)
        nlayer += 1
        _lorder = []
        _Contract221_list = []
        for idx in lorder:
            for x in adj[:, idx]:
                if flag[x] == 0:

                    counts = 0
                    for xx in adj[:, x]:
                        if flag[xx] == nlayer - 1:
                            counts += 1
                    if counts == 2:
                        _Contract221_list.append(x)
                        for xx in adj[:, x]:
                            if flag[xx] == 0:
                                _lorder.append(xx)
                                flag[xx] = nlayer
                                flag[x] = -nlayer + 1
                    else:
                        _lorder.append(x)
                        flag[x] = nlayer
        Contract221_list.append(_Contract221_list)

        n_l = np.size(_lorder)
        #         print("rank =",n_l)
        for i in range(n_l):
            bonds = 0
            for x in adj[:, _lorder[i]]:
                if flag[x] == 0:
                    _lorder.append(x)
                    flag[x] = nlayer
        lorder = _lorder
    # sort layerwise list
    for i in range(len(Contract_list)):
        llist = Contract_list[i]
        lflag = [0 for i in range(n)]
        lflag[llist[0]] = 1
        for x in adj[:, llist[0]]:
            if flag[x] == i + 1:
                start = x
                lflag[x] = 1
                break
        _llist = [llist[0], start]
        while 1:
            ok = 0
            for x in adj[:, start]:
                if (flag[x] == i + 1) & (lflag[x] == 0):
                    start = x
                    lflag[x] = 1
                    ok = 1
                    _llist.append(start)
                    break
            if ok == 0:
                break
        Contract_list[i] = _llist
        Points = []
        for i in range(len(Contract_list)):
            for x in Contract_list[i]:
                Points.append(x)
            for x in Contract221_list[i]:
                Points.append(x)
    return Contract_list, Contract221_list, Points, flag

In [6]:
def pos(x, l):
    for i in range(len(l)):
        if l[i] == x:
            break
    return i

In [7]:
# currently only work for system without Contract221_list
def sort_loop_contraction(Contract_list, adj, flag):
    bond_list = Contract_list[0].copy()
    bond_list.reverse()
    ops = [[(0, 1), (0, 1), (0, 1), (1, 2)]]
    for i in range(1, len(Contract_list)):
        if i != 1:
            bond_list_tmp = bond_list.copy()
            for xx in range(len(bond_list)):
                bond_list[xx] = bond_list_tmp[(xx - 1) % len(bond_list)]
        #         print("bl=",bond_list)

        trylist = []
        for x in Contract_list[i]:
            for idx in adj[:, x]:
                if flag[idx] == i:
                    trylist.append(idx)
        Contract_list_corrected = Contract_list[i].copy()
        for x in range(len(trylist)):
            if pos(trylist[x], bond_list) % len(trylist) != (
                pos(trylist[(x + 1) % len(trylist)], bond_list) - 1
            ) % len(trylist):
                for xx in range(len(Contract_list[i])):
                    Contract_list_corrected[xx] = Contract_list[i][-xx]
                break

        for idx in adj[:, bond_list[0]]:

            if flag[idx] == i + 1:
                shift = pos(idx, Contract_list_corrected)
                Contract_list_corrected_tmp = Contract_list_corrected.copy()
                for xx in range(len(Contract_list_corrected_tmp)):
                    Contract_list_corrected[xx] = Contract_list_corrected_tmp[
                        (shift + xx) % len(Contract_list_corrected_tmp)
                    ]

        Contract_list[i] = Contract_list_corrected.copy()

        bond_list = []
        ops_ = [(0, 1)]
        idxx = 0
        #         print(Contract_list_corrected)
        for x in Contract_list_corrected[1:]:
            idxx += 1
            connected = 0
            for idx in adj[:, x]:

                if flag[idx] == i:
                    connected += 1
            if idxx == (len(Contract_list_corrected) - 1):
                if connected == 0:
                    bond_list.append(x)
                    ops_.append((1, 2))

                if connected == 1:
                    ops_.append((2, 3))
            else:
                if connected == 0:
                    bond_list.append(x)
                    ops_.append((0, 1))
                if connected == 1:
                    ops_.append((1, 2))
        ops.append(ops_)
    return ops

In [8]:
for filename in [
    "C20-Ih.xyz",
    "C60-Ih.xyz",
    "C180-0.xyz",
    "C240-0.xyz",
    "C320-0.xyz",
    "C500-0.xyz",
    "C540-0.xyz",
    "C720-0.xyz",
]:
    n, coo, adj = xyz_read(filename)
    Contract_list, Contract221_list, Points, flag = find_loop_contraction(
        adj, find_polygen(adj, 5)
    )
    ops = sort_loop_contraction(Contract_list, adj, flag)
    f = open("C" + str(n) + ".path", "w")
    rank = 3
    f.write(str(n) + "\n")
    for x in ops:
        for op in x:
            f.write(str(op[0]) + " " + str(op[1]) + "\n")
#             if op[1] == 2:
#                 rank -= 1
#             if op[1] == 1:
#                 rank += 1
#             if op[1] == 3:
#                 rank -= 3
    f.close()